<a href="https://colab.research.google.com/github/OsamaAwniHamdan/Prediction-of-Product-Sales/blob/main/Project_part_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project - Part 6
  - Osama Hamdan

In [1]:

# Mount google drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## Mount Google drive

In [2]:
import pandas as pd

sales_file_path = '/content/drive/MyDrive/CodingDojo/01-Fundamentals/Week02/Data/sales_predictions_2023.csv'

df = pd.read_csv(sales_file_path)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


##  Preprossing the data set

In [3]:
df.duplicated(keep=False).sum()

0

- No duplicates

### Check columns

In [4]:
cat_cols = df.select_dtypes(include='object').columns
df[cat_cols].value_counts()

Item_Identifier  Item_Fat_Content  Item_Type              Outlet_Identifier  Outlet_Size  Outlet_Location_Type  Outlet_Type      
DRA12            LF                Soft Drinks            OUT035             Small        Tier 2                Supermarket Type1    1
FDV27            Regular           Meat                   OUT019             Small        Tier 1                Grocery Store        1
FDV32            Low Fat           Fruits and Vegetables  OUT035             Small        Tier 2                Supermarket Type1    1
                                                          OUT019             Small        Tier 1                Grocery Store        1
                                                          OUT018             Medium       Tier 3                Supermarket Type2    1
                                                                                                                                    ..
FDJ33            Regular           Snack Foods            OU

- Fix inconsistency

In [6]:
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace({'LF': 'Low Fat', 'reg': 'Regular', 'low fat': 'Low Fat' })


- Check numeric columns

In [7]:
df.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


- Check Objects columns

In [8]:
df.describe(include='object')

,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
count,8523,8523,8523,8523,6113,8523,8523
unique,1559,2,16,10,3,3,4
top,FDW13,Low Fat,Fruits and Vegetables,OUT027,Medium,Tier 3,Supermarket Type1
freq,10,5517,1232,935,2793,3350,5577


- I will drop the Item_Identifier and Item_type because of **high credntality**




## EDA Functions
  - functions for preprossing and apply models

In [9]:
from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd

set_config(transform_output='pandas')

def preprocess_numaric(numaric_cols):
  num_imputer = SimpleImputer(strategy='median')
  scaler = StandardScaler()
  num_pip = make_pipeline(num_imputer, scaler)
  return ('num', num_pip, numaric_cols)

def prprocess_categorical(categorical_cols):
  cat_imputer = SimpleImputer(strategy='most_frequent')
  one_het = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
  cat_pip = make_pipeline(cat_imputer, one_het)
  return ('cat', cat_pip, categorical_cols)

def prepocess_ordinal(ord_orders, ordinal_cols):
  ord_imputer = SimpleImputer(strategy='most_frequent')
  ord_encoder = OrdinalEncoder(categories=ord_orders)
  scaler = StandardScaler()
  ord_pip = make_pipeline(ord_imputer, ord_encoder, scaler)
  return ('ordinal', ord_pip, ordinal_cols)

def preprocess_all(numeric_cols=None, categorical_cols=None, ord_cols=None, ord_vals=[]):
  process = []
  if numeric_cols:
    process.append(preprocess_numaric(numeric_cols))
  if categorical_cols:
    process.append(prprocess_categorical(categorical_cols))
  if ord_cols:
    process.append(prepocess_ordinal(ord_vals, ord_cols))
  return ColumnTransformer(process, verbose_feature_names_out=False)

def split_data(df, target, features, test_size=0.25, random_state=42):
  X = df[features]
  y = df[target].values.ravel()
  return train_test_split(X, y, test_size=test_size, random_state=random_state)

def regression_metrics(y_true, y_pred, label='', verbose = True, output_dict=False):
  # Get metrics
  mae = mean_absolute_error(y_true, y_pred)
  mse = mean_squared_error(y_true, y_pred)
  rmse = mean_squared_error(y_true, y_pred, squared=False)
  r_squared = r2_score(y_true, y_pred)
  if verbose == True:
    # Print Result with Label and Header
    header = "-"*60
    print(header, f"Regression Metrics: {label}", header, sep='\n')
    print(f"- MAE = {mae:,.3f}")
    print(f"- MSE = {mse:,.3f}")
    print(f"- RMSE = {rmse:,.3f}")
    print(f"- R^2 = {r_squared:,.3f}")
  if output_dict == True:
      metrics = {'Label':label, 'MAE':mae,
                 'MSE':mse, 'RMSE':rmse, 'R^2':r_squared}
      return metrics


def evaluate_regression(reg, X_train, y_train, X_test, y_test, verbose = True,
                        output_frame=False):
  # Get predictions for training data
  y_train_pred = reg.predict(X_train)

  # Call the helper function to obtain regression metrics for training data
  results_train = regression_metrics(y_train, y_train_pred, verbose = verbose,
                                     output_dict=output_frame,
                                     label='Training Data')
  print()
  # Get predictions for test data
  y_test_pred = reg.predict(X_test)
  # Call the helper function to obtain regression metrics for test data
  results_test = regression_metrics(y_test, y_test_pred, verbose = verbose,
                                  output_dict=output_frame,
                                    label='Test Data' )

  # Store results in a dataframe if ouput_frame is True
  if output_frame:
    results_df = pd.DataFrame([results_train,results_test])
    # Set the label as the index
    results_df = results_df.set_index('Label')
    # Set index.name to none to get a cleaner looking result
    results_df.index.name=None
    # Return the dataframe
    return results_df.round(3)



def apply_model(model, preprocesser, X_train, X_test, y_train, y_test):
  model_pip = make_pipeline(preprocesser, model)
  model_pip.fit(X_train, y_train)
  evaluate_regression(model_pip,  X_train, y_train, X_test, y_test, verbose = True,
                        output_frame=False)
  return model_pip




## Splitting data set and dropping unrequierd columns

In [10]:
target = ['Item_Outlet_Sales']
features = df.drop(columns=['Item_Outlet_Sales', 'Item_Identifier', 'Item_Type']).columns.tolist()
numeric_cols = df[features].select_dtypes(include='number').columns.tolist()
ordinal_cols = ['Outlet_Size']
cat_cols = df[features].select_dtypes(include='object').columns.drop(ordinal_cols).tolist()

X_train, X_test, y_train, y_test = split_data(df, target, features)



### Preprocess data using

In [11]:

ord_orders = [['Small', 'Medium', 'High']]

preprocess = preprocess_all(numeric_cols=numeric_cols, categorical_cols=cat_cols, ord_cols=ordinal_cols, ord_vals=ord_orders)
preprocess

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['Item_Weight', 'Item_Visibility', 'Item_MRP',
                                  'Outlet_Establishment_Year']),
                                ('cat',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='igno...
                                                                sparse_output=False))]),
                                 ['Item_Fat_Content', 'Outlet_Identifier',
                                  'Outlet_Location_Type', 'Outlet_Type']),
                                ('ordinal',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Small',
                                                                              'Medium',
                                                                              'High']])),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['Outlet_Size'])],
                  verbose_feature_names_out=False)

## Linear Regression model

In [13]:
from sklearn.linear_model import LinearRegression

In [14]:
linreg = LinearRegression()
apply_model(linreg, preprocess, X_train, X_test, y_train, y_test)

------------------------------------------------------------
Regression Metrics: Training Data
------------------------------------------------------------
- MAE = 848.087
- MSE = 1,299,826.729
- RMSE = 1,140.099
- R^2 = 0.561

------------------------------------------------------------
Regression Metrics: Test Data
------------------------------------------------------------
- MAE = 802.472
- MSE = 1,187,071.833
- RMSE = 1,089.528
- R^2 = 0.570


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['Item_Weight',
                                                   'Item_Visibility',
                                                   'Item_MRP',
                                                   'Outlet_Establishment_Year']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencode...
                                                  ['Item_Fat_Content',
                                                   'Outlet_Identifier',
                                                   'Outlet_Location_Type',
                                                   'Outlet_Type']),
                                                 ('ordinal',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ordinalencoder',
                                                                   OrdinalEncoder(categories=[['Small',
                                                                                               'Medium',
                                                                                               'High']])),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['Outlet_Size'])],
                                   verbose_feature_names_out=False)),
                ('linearregression', LinearRegression())])

- The RMSE Error for both training and testing set is arround **1100** which considered as a high error rate in compare of the Mean value: **2181.288914**

The Model is **underfitting**

## Random forest Model

In [15]:
from sklearn.ensemble import RandomForestRegressor

rfreg = RandomForestRegressor(random_state=42)
model = apply_model(rfreg,preprocess, X_train, X_test, y_train, y_test)

------------------------------------------------------------
Regression Metrics: Training Data
------------------------------------------------------------
- MAE = 297.199
- MSE = 183,571.772
- RMSE = 428.453
- R^2 = 0.938

------------------------------------------------------------
Regression Metrics: Test Data
------------------------------------------------------------
- MAE = 771.166
- MSE = 1,218,402.508
- RMSE = 1,103.813
- R^2 = 0.558


- By looking to error metrics the training data doing very good while the testing data metrics is not as well.
- The model is **overfitting**

### Tune the model

In [16]:
model.get_params()

{'memory': None,
 'steps': [('columntransformer',
   ColumnTransformer(transformers=[('num',
                                    Pipeline(steps=[('simpleimputer',
                                                     SimpleImputer(strategy='median')),
                                                    ('standardscaler',
                                                     StandardScaler())]),
                                    ['Item_Weight', 'Item_Visibility', 'Item_MRP',
                                     'Outlet_Establishment_Year']),
                                   ('cat',
                                    Pipeline(steps=[('simpleimputer',
                                                     SimpleImputer(strategy='most_frequent')),
                                                    ('onehotencoder',
                                                     OneHotEncoder(handle_unknown='igno...
                                                                   sparse_output=Fal

In [17]:
params = {'randomforestregressor__n_estimators':[100, 200, 300],
          'randomforestregressor__max_depth':[10, 20],
          'randomforestregressor__min_samples_split':[2, 5],
          'randomforestregressor__min_samples_leaf':[1, 2]}

In [18]:
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(model, params, n_jobs = -1, verbose = 1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         ['Item_Weight',
                                                                          'Item_Visibility',
                                                                          'Item_MRP',
                                                                          'Outlet_Establishment_Year']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='most_fre...
                                                                                          StandardScaler())]),
                                                                         ['Outlet_Size'])],
                                                          verbose_feature_names_out=False)),
                                       ('randomforestregressor',
                                        RandomForestRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid={'randomforestregressor__max_depth': [10, 20],
                         'randomforestregressor__min_samples_leaf': [1, 2],
                         'randomforestregressor__min_samples_split': [2, 5],
                         'randomforestregressor__n_estimators': [100, 200,
                                                                 300]},
             verbose=1)

In [19]:
grid_search.best_params_

{'randomforestregressor__max_depth': 10,
 'randomforestregressor__min_samples_leaf': 1,
 'randomforestregressor__min_samples_split': 2,
 'randomforestregressor__n_estimators': 100}

In [20]:
evaluate_regression(grid_search.best_estimator_, X_train, y_train, X_test, y_test, verbose = True,
                        output_frame=False)

------------------------------------------------------------
Regression Metrics: Training Data
------------------------------------------------------------
- MAE = 638.429
- MSE = 816,223.447
- RMSE = 903.451
- R^2 = 0.724

------------------------------------------------------------
Regression Metrics: Test Data
------------------------------------------------------------
- MAE = 737.658
- MSE = 1,127,274.457
- RMSE = 1,061.732
- R^2 = 0.591


**Better performance after tuning, but still we have underfitting.**



- Overall, the best model is definitely the tuned Random Forest Regressor Model.
- For Tuned random forest the R2 = **0.591** which means that approximately 59.1% of the variation in the data can be explained by the model.
- RMSE = **1,061.732**, that means the model's predictions are off by approximately 1,061.732$ which is a moderate prediction compared with the mean values.
- Compring trining to testing showing that the model is underfitting.
